In [1]:
import numpy as np
from scipy.optimize import minimize_scalar

In [2]:
# from parepy_toolbox.distributions import non_normal_approach_normal

# params_normal = {'loc': 0, 'scale': 1}
# result_normal = non_normal_approach_normal(1.3065, 'normal', params_normal)
# print(result_normal)

# params_gumbel_max = {'loc': 0.887487, 'scale': 5.1302}
# result_gumbel_max = non_normal_approach_normal(1.3065, 'gumbel max', params_gumbel_max)
# print(result_gumbel_max)

# params_log_normal = {'loc': 0.887487, 'scale': 5.1302}
# result_log_normal = non_normal_approach_normal(1.3065, 'log normal', params_log_normal)
# print(result_log_normal)

In [3]:
def obj(x, none_variable):
    simga_y = x[0]
    w = x[1]
    m = x[2]
    return simga_y * w - m

def grad_obj(x, none_variable):
    sigma_y = x[0]
    w = x[1]
    m = x[2]
    grad_g = [w, sigma_y, -1]
    return grad_g

def deterministic_algorithm_structural_analysis(setup: dict):
    
    mu = [40., 50., 1000.]
    sigma = [5., 2.5, 200.]
    tol = 1e-6
    max_iter = 20
    beta_list = [10000]
    error = 1000
    iter = 0
    step = 1

    x = np.transpose(np.array([mu.copy()]))
    mu = x.copy()
    jacobian_xy = np.diag(sigma)
    jacobian_xy_trans = np.transpose(jacobian_xy)
    jacobian_yx = np.linalg.inv(jacobian_xy)
    y = jacobian_yx @ (x - mu)
    x = jacobian_xy @ y + mu

    while (error > tol and iter < max_iter):
        beta = np.linalg.norm(y)
        beta_list.append(beta)
        g_y = obj(x.flatten().tolist(), None)
        grad_g_x = grad_obj(x.flatten().tolist(), None)
        grad_g_y = np.dot(jacobian_xy_trans, np.transpose(np.array([grad_g_x])))
        num = (np.transpose(grad_g_y) @ y - g_y)
        norm = np.linalg.norm(grad_g_y)
        norm2 = norm ** 2
        #alpha = grad_g_y / norm
        #aux = g_y / norm
        #y = -alpha * (beta + aux)
        print(grad_g_y, y, g_y, num, norm2)
        d = num / norm2 @ grad_g_y - y
        y += + step * d
        error = np.abs(beta_list[iter + 1] - beta_list[iter])
        iter += 1
        x = jacobian_xy @ y + mu
    return beta

In [4]:
# Dataset
f = {'type': 'normal', 'parameters': {'mean': 40, 'sigma': 5.}, 'stochastic variable': False}
p = {'type': 'normal', 'parameters': {'mean': 50, 'sigma': 2.5}, 'stochastic variable': False}
w = {'type': 'normal', 'parameters': {'mean': 1000, 'sigma': 200}, 'stochastic variable': False}
var = [f, p, w]

# PAREpy setup
setup = {
                'tolerance': 1e-6, 
                'max iterations': 20,
                'numerical model': 'fosm', 
                'variables settings': var, 
                'number of state limit functions or constraints': 1, 
                'none variable': None,
                'objective function': obj,
                'gradient objective function': grad_obj,
                'name simulation': None,
        }

# Call algorithm
beta = deterministic_algorithm_structural_analysis(setup)
print(beta)

[[ 250.]
 [ 100.]
 [-200.]] [[0.]
 [0.]
 [0.]] 1000.0 [[-1000.]] 112500.00000000001


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 1)